# Simulations using NFBC1966 results

Using chromosome 21 of the NFBC (Stampeed) data ($n=5340$ samples and $p = 5540$ SNPs), let us simulate:
+ Non-independent samples
+ Polygenic traits where every SNP contributes to the variance of the phenotypes, but only $k$ SNPs contribute to the mean of the phenotype.

## Simulation model

For $r$ traits, our simulation model is:

$$\mathbf{Y}_{r \times n} \sim \text{MatrixNormal}(\mathbf{B}_{r \times p}\mathbf{X}_{p \times n}, \ \ \Sigma_{r \times r} , \ \ \sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n})$$

+ $\mathbf{X}_{p \times n}$ contains *all* predictors (genetic + non-genetic)
+ $\mathbf{B}_{r \times p}$ contains (true) regression coefficients. $k$ entries are non-zero
+ $\Phi$ is the GRM estimated from genotypes
+ $\Sigma_{r \times r} = \mathbf{A}^t\mathbf{A}$ is the row (trait) covariance matrix where $A_{ij} \sim N(0, 1)$
+ $\sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n}$ is the column (sample) covariance matrix
+ $\sigma_g^2 = 0.6$ and $\sigma_e^2 = 0.4$ (thus heritability is 60%)
+ $k = \{10, 20, 50, 100\}$ (number of causal SNPs)
+ Pleiotripic overlap = 20% (e.g. if $k=10$, then 2 causal SNPs are shared in each trait, thus only 8 unique causal SNPs)
+ $\beta_i \sim N(0, 0.1)$
+ $r = \{2, 5\}$  (number of traits)
+ $n=5340$ samples
+ $p = 5540$ SNPs

In [1]:
# using Revise
# using MendelIHT
# using SnpArrays
# using Random
# using GLM
# using DelimitedFiles
# using Distributions
# using LinearAlgebra
# using CSV
# using DataFrames
# using StatsBase
# using Statistics
# using TraitSimulation

using Distributed
addprocs(8)
@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
    using StatsBase
    using Statistics
    using TraitSimulation
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1317


### Helper functions for parsing results

In [5]:
"""
Find the `nsnps` most significant SNPs for mvPLINK in simulation `sim`.
"""
function get_top_mvPLINK_SNPs(sim::Int, nsnps::Int)
    n, p = 5402, 26906
    dir = "NFBCsim/sim$sim/"
    
    # read mvPLINK result
    mvplink_df = CSV.read(dir * "plink.mqfam.total", DataFrame, delim=' ', ignorerepeated=true)

    # get pvalues, possibly accounting for "NA"s
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end
    perm = sortperm(pval)
    return perm[1:nsnps]
end

"""
Find the position of the `nsnps` most significant SNPs for GEMMA in simulation `sim`.
"""
function get_top_GEMMA_SNP_ids(sim::Int, nsnps::Int)
    dir = "NFBCsim/sim$sim/"
    gemma_df = CSV.read(dir * "gemma.sim$sim.assoc.txt", DataFrame)
    pval_wald = gemma_df[!, :p_wald]
    perm = sortperm(pval_wald)
    return perm[1:nsnps]
end

"""
Find significant SNPs return by IHT in simulation `sim`.
"""
function get_IHT_SNPs(sim::Int)
    dir = "NFBCsim/sim$sim/"
    iht_β1 = vec(readdlm(dir * "iht_beta1.txt"))
    iht_β2 = vec(readdlm(dir * "iht_beta2.txt"))
    detected_snps = findall(!iszero, iht_β1) ∪ findall(!iszero, iht_β2)
    return unique(detected_snps)
end

"""
Get positions for the truly causal SNPs in simulation `sim`. 
"""
function get_true_SNPs(sim::Int)
    model = "NFBCsim"
    dir = "NFBCsim/sim$sim/"
    trueB = readdlm(dir * "trueb.txt")
    causal_snps = unique([x[1] for x in findall(!iszero, trueB)])
    return causal_snps
end

"""
Get causal SNPs' position in GEMMA's result for simulation `sim`. Note gemma have snp filtering.
"""
function get_gemma_causal_snp_pos(sim::Int)
    model = "NFBCsim"
    nfbc = SnpData("/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/imputed_with_0/NFBC_imputed_with_0")
    dir = "NFBCsim/sim$sim/"
    trueB = readdlm(dir * "trueb.txt")
    causal_snps = unique([x[1] for x in findall(!iszero, trueB)])
    causal_snp_rsID = nfbc.snp_info.snpid[causal_snps]
    gemma_df = CSV.read("NFBCsim/sim$sim/gemma.sim$sim.assoc.txt", DataFrame)
    gemma_snps = gemma_df[!, :rs]
    causal_snp_idx = convert(Vector{Int}, indexin(causal_snp_rsID, gemma_snps))
    
    # also need IHT's selected SNPs
    iht_snps_rsID = nfbc.snp_info.snpid[get_IHT_SNPs(sim)]
    iht_snps_idx = convert(Vector{Int}, indexin(iht_snps_rsID, gemma_snps))
    
    # also need SNP positions in GEMMA dataframe
    gemma2nfbc_idx = convert(Vector{Int}, indexin(gemma_snps, nfbc.snp_info.snpid))
    gemma_snp_pos = Vector{Int}(undef, size(gemma_df, 1))
    for i in 1:size(gemma_df, 1)
        gemma_snp_pos[i] = nfbc.snp_info.position[gemma2nfbc_idx[i]]
    end
    insertcols!(gemma_df, size(gemma_df, 2) + 1, :pos => gemma_snp_pos)
    
    return gemma_df, causal_snp_idx, iht_snps_idx
end

"""
- filename: gemma's output file name
- correct_snps: indices for real causal SNPs

returns: power, number of false positives, and false positive rate
"""
function process_gemma_result(filename, correct_snps)
    # read GEMMA result
    gemma_df = CSV.read(filename, DataFrame)
    snps = size(gemma_df, 1)

    # pvalues
    pval_wald = gemma_df[!, :p_wald]
#    pval_lrt = gemma_df[!, :p_lrt]
#     pval_score = gemma_df[!, :p_score]

    # estimated beta
    estim_β1 = gemma_df[!, :beta_1]
    estim_β2 = gemma_df[!, :beta_2]

    # estimated covariance matrix
    estim_σ11 = gemma_df[!, :Vbeta_1_1]
    estim_σ12 = gemma_df[!, :Vbeta_1_2]
    estim_σ22 = gemma_df[!, :Vbeta_2_2];

    # check how many real SNPs were recovered
    signif_snps = findall(x -> x ≤ 0.05 / snps, pval_wald) # gemma's selected snps
    signif_snps_rsID = gemma_df[signif_snps, :rs]
    
    # compute power, false positives, and false positive rate
    power_and_fpr(snps, correct_snps, signif_snps_rsID)
end

"""
- filename: mvPLINK's output file name
- correct_snps: indices for real causal SNPs

returns: power, number of false positives, and false positive rate
"""
function process_mvPLINK(filename, correct_snps)
    # read mvPLINK result
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    snps = size(mvplink_df, 1)

    # get pvalues, possibly accounting for "NA"s
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end

    # SNPs passing threshold
    signif_snps = findall(x -> x ≤ 0.05 / snps, pval)

    # compute power, false positives, and false positive rate
    power_and_fpr(snps, correct_snps, signif_snps)
end

"""
Computes power and false positive rates
- p: total number of SNPs
- correct_snps: Indices (or rsID) of the true causal SNPs
- detected_snps: Indices (or rsID) of SNPs that are significant after testing
"""
function power_and_fpr(p::Int, correct_snps::Vector, detected_snps::Vector)
    power = length(detected_snps ∩ correct_snps) / length(correct_snps)
    FP = length(detected_snps) - length(detected_snps ∩ correct_snps) # number of false positives
    TN = p - length(detected_snps) # number of true negatives
    FPR = FP / (FP + TN)
    return power, FP, FPR
end

"""
Summarize all simulations for IHT, mvPLINK, GEMMA in computation time, true positives,
false positives, and false positive rates. 
"""
function summarize_repeats()
    model = "NFBCsim"
    n, p = 5402, 26906
    sims = 1:50 # k = 10, r = 2, βoverlap=2, polygenic model
    nfbc = SnpData("/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/imputed_with_0/NFBC_imputed_with_0")
    snp_rsID = nfbc.snp_info.snpid

    iht_time, iht_power, iht_FP, iht_FPR = Float64[], Float64[], Float64[], Float64[]
    mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPR = Float64[], Float64[], Float64[], Float64[]
    gemma_time, gemma_power, gemma_FP, gemma_FPR = Float64[], Float64[], Float64[], Float64[]

    for sim in sims
        dir = "NFBCsim/sim$sim/"
        try
            # correct SNPs
            trueB = readdlm(dir * "trueb.txt")
            causal_snps = unique([x[1] for x in findall(!iszero, trueB)])
            causal_snps_rsID = snp_rsID[causal_snps]

            # IHT
            iht_β1 = vec(readdlm(dir * "iht_beta1.txt"))
            iht_β2 = vec(readdlm(dir * "iht_beta2.txt"))
            detected_snps = findall(!iszero, iht_β1) ∪ findall(!iszero, iht_β2)
            ihtpower, ihtFP, ihtFPR = power_and_fpr(p, causal_snps, detected_snps)

            # MVPLINK
            plinkpower, plinkFP, plinkFPR = process_mvPLINK(dir * "plink.mqfam.total", causal_snps)

            # GEMMA 
            gemmapower, gemmaFP, gemmaFPR = process_gemma_result(dir * "gemma.sim$sim.assoc.txt", causal_snps_rsID)
            
            push!(iht_power, ihtpower); push!(iht_FP, ihtFP); push!(iht_FPR, ihtFPR); 
            push!(mvPLINK_power, plinkpower); push!(mvPLINK_FP, plinkFP); push!(mvPLINK_FPR, plinkFPR); 
            push!(gemma_power, gemmapower); push!(gemma_FP, gemmaFP); push!(gemma_FPR, gemmaFPR);
        catch
            println("simulation $sim failed!")
        end
    end

    return iht_time, iht_power, iht_FP, iht_FPR,
        mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPR,
        gemma_time, gemma_power, gemma_FP, gemma_FPR
end

"""
Imports gemma p-values, causal SNPs, and IHT selected SNP, and plot manhattan plot using MendelPlots.jl
"""
function plot_gemma_manhattan(sim::Int)
    # GEMMA causal SNPs
    gemma_df, causal_snps, iht_snps = get_gemma_causal_snp_pos(sim)
    rename!(gemma_df, [:p_wald => :pval, :rs => :snpid])
    gemma_df[findall(x -> x < 1e-50, gemma_df[!, :pval]), :pval] .= 1e-50
    empty_col = ["" for i in 1:size(gemma_df, 1)]
    insertcols!(gemma_df, size(gemma_df, 2) + 1, :empty_col => empty_col)

    manhattan(gemma_df, outfile = "NFBCsim/manhattan_gemma_sim$sim.png",
        annotateinds = causal_snps, annotateinds2 = iht_snps,
        annotatevar=:empty_col, titles="GEMMA simulation $sim")
    display("image/png", read("NFBCsim/manhattan_gemma_sim$sim.png"))
end

"""
Imports mvPLINK p-values, causal SNPs, and IHT selected SNP, and plot manhattan plot using MendelPlots.jl
"""
function plot_mvPLINK_manhattan(sim::Int)
    # mvPLINK
    filename = "NFBCsim/sim$sim/plink.mqfam.total"
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end
    pval[findall(x -> x < 1e-50, pval)] .= 1e-50 # limit smallest pvalues

    # causal SNPs
    causal_snps = get_true_SNPs(sim)
    
    # IHT SNPs
    iht_snps = get_IHT_SNPs(sim)

    # make dataframe to input into MendelPlots
    snpdata = SnpData("/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/imputed_with_0/NFBC_imputed_with_0")
    rename!(snpdata.snp_info, [:chr, :snpid, :genetic_distance, :pos, :allele1, :allele2])
    insertcols!(snpdata.snp_info, size(snpdata.snp_info, 2) + 1, :pval => pval)
    empty_col = ["" for i in 1:size(snpdata.snp_info, 1)]
    insertcols!(snpdata.snp_info, size(snpdata.snp_info, 2) + 1, :empty_col => empty_col)

    # plot
    manhattan(snpdata.snp_info, outfile = "NFBCsim/manhattan_mvPLINK_sim$sim.png",
        annotateinds = causal_snps, annotateinds2 = iht_snps, 
        annotatevar=:empty_col, titles="mvPLINK simulation $sim")
    display("image/png", read("NFBCsim/manhattan_mvPLINK_sim$sim.png"))
end

"""
Imports mvPLINK p-values and plot QQ plot using MendelPlots.jl
"""
function plot_mvPLINK_QQ(sim::Int)
    filename = "NFBCsim/sim$sim/plink.mqfam.total"
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end
    pval[findall(x -> x < 1e-50, pval)] .= 1e-50 # limit smallest pvalues
    qq(pval, outfile = "NFBCsim/QQ_mvPLINK_sim$sim.png",
        ylabel="mvPLINK observed -log10(p)", titles="mvPLINK simulation $sim")
    display("image/png", read("NFBCsim/QQ_mvPLINK_sim$sim.png"))
end

"""
Imports gemma p-values and plot QQ plot using MendelPlots.jl
"""
function plot_gemma_QQ(sim::Int)
    filename = "NFBCsim/sim$sim/gemma.sim$sim.assoc.txt"
    gemma_df = CSV.read(filename, DataFrame)
    pval_wald = gemma_df[!, :p_wald]
    pval_wald[findall(x -> x < 1e-50, pval_wald)] .= 1e-50 # limit smallest pvalues
    qq(pval_wald, outfile = "NFBCsim/QQ_gemma_sim$sim.png",
        ylabel="GEMMA observed -log10(p)", titles="GEMMA simulation $sim")
    display("image/png", read("NFBCsim/QQ_gemma_sim$sim.png"))
end

"""
After performing n simulations using `run_repeats`, this function reads the summary files
for each simulation and summarizes the result. 
"""
function read_summary(simulations::UnitRange)
    model = "NFBCsim"
    
    iht_time, iht_power, iht_FP, iht_FPT = Float64[], Float64[], Float64[], Float64[]
    mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT = Float64[], Float64[], Float64[], Float64[]
    gemma_time, gemma_power, gemma_FP, gemma_FPT = Float64[], Float64[], Float64[], Float64[]

    regex = r"= (\d+\.\d+) seconds, power = (\d+\.\d+), FP = (\d+), FPR = (\d\.\d+e?-?\d*)"
    for sim in simulations
        if !isdir("$model/sim$sim") || !isfile("$model/sim$(sim)/summary.txt")
            println("Simulation $sim failed!")
            continue
        end
        try
            open("$model/sim$(sim)/summary.txt", "r") do io
                readline(io); readline(io); readline(io)

                # parse IHT result
                iht = match(regex, readline(io))
                push!(iht_time, parse(Float64, iht[1]))
                push!(iht_power, parse(Float64, iht[2]))
                push!(iht_FP, parse(Float64, iht[3]))
                push!(iht_FPT, parse(Float64, iht[4]))

                # parse mvPLINK result
                mvPLINK = match(regex, readline(io))
                push!(mvPLINK_time, parse(Float64, mvPLINK[1]))
                push!(mvPLINK_power, parse(Float64, mvPLINK[2]))
                push!(mvPLINK_FP, parse(Float64, mvPLINK[3]))
                push!(mvPLINK_FPT, parse(Float64, mvPLINK[4]))

                # parse mvPLINK result
                gemma = match(regex, readline(io))
                push!(gemma_time, parse(Float64, gemma[1]))
                push!(gemma_power, parse(Float64, gemma[2]))
                push!(gemma_FP, parse(Float64, gemma[3]))
                push!(gemma_FPT, parse(Float64, gemma[4]))
            end
        catch
            println("Simulation $sim failed!")
            continue
        end
    end

    # save summary statistics
    open("$(model)_summary.txt", "w") do io
        println(io, "iht_time,iht_power,iht_FP,iht_FPT,mvPLINK_time,mvPLINK_power," * 
            "mvPLINK_FP,mvPLINK_FPT,gemma_time,gemma_power,gemma_FP,gemma_FPT")
        for i in eachindex(iht_time)
            print(io, iht_time[i], ',', iht_power[i], ',', iht_FP[i], ',', iht_FPT[i], ',')
            print(io, mvPLINK_time[i], ',', mvPLINK_power[i], ',', mvPLINK_FP[i], ',', mvPLINK_FPT[i], ',')
            print(io, gemma_time[i], ',', gemma_power[i], ',', gemma_FP[i], ',', gemma_FPT[i], "\n")
        end
    end
    
    return iht_time, iht_power, iht_FP, iht_FPT, 
        mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT,
        gemma_time, gemma_power, gemma_FP, gemma_FPT
end

"""
k = Number of causal SNPs
p = Total number of SNPs
traits = Number of traits (phenotypes)
overlap = number of causal SNPs shared in each trait
"""
function simulate_random_beta(k::Int, p::Int, traits::Int; overlap::Int=0, βσ=1.0)
    d = Normal(0, βσ)
    true_b = zeros(p, traits)
    if overlap == 0
        causal_snps = sample(1:(traits * p), k, replace=false)
        true_b[causal_snps] = rand(d, k)
    else
        shared_snps = sample(1:p, overlap, replace=false)
        weight_vector = aweights(1 / (traits * (p - overlap)) * ones(traits * p))
        for i in 1:traits
            weight_vector[i*shared_snps] .= 0.0 # avoid sampling from shared snps
        end
        @assert sum(weight_vector) ≈ 1.0
        # simulate β for shared predictors
        for i in 1:traits
            true_b[shared_snps, i] = rand(d, overlap)
        end
        # simulate β for none shared predictors
        nonshared_snps = sample(1:(traits * p), weight_vector, k - (traits * overlap), replace=false)
        true_b[nonshared_snps] = rand(d, k - (traits * overlap))
    end

    return true_b
end

"""
ad-hoc matrix-matrix multiplication (out = st * v) using SnpLinAlg
"""
function adhoc_mul!(
    out::AbstractMatrix{T}, 
    st::SnpLinAlg{T},
    v::AbstractMatrix{T}) where T <: AbstractFloat
    @assert size(out, 1) == size(st, 1) && size(v, 2) == size(v, 2) && size(st, 2) == size(v, 1)
    for i in 1:size(v, 2)
        outi = @view(out[:, i])
        vi = @view(v[:, i])
        SnpArrays.mul!(outi, st, vi)
    end
end

function simulate_NFBC1966_polygenic(
    plinkname::String, k::Int, r::Int;
    seed::Int=2021, σg=0.6, σe=0.4, βoverlap=2, βσ=1.0,
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = SnpArray(plinkname * ".bed")
#     xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, impute=true, center=true, scale=true)
    xla = convert(Matrix{Float64}, x, impute=true, center=true, scale=true)
    n, p = size(x)
    
    # intercept is the only nongenetic covariate
    Z = ones(n, 1)
    intercepts = randn(r)' # each trait have different intercept

    # simulate β
    B = simulate_random_beta(k, p, r, overlap=βoverlap, βσ=βσ)

    # between trait covariance matrix
    Σ = random_covariance_matrix(r)

    # between sample covariance is identity + GRM
    Φ = grm(x, method=:Robust)
    V = σg * Φ + σe * I

    # simulate y using TraitSimulations.jl (https://github.com/OpenMendel/TraitSimulation.jl/blob/master/src/modelframework.jl#L137)
    vc = @vc Σ ⊗ V
    μ = zeros(n, r)
    μ_null = zeros(n, r)
    LinearAlgebra.mul!(μ_null, Z, intercepts)
#     adhoc_mul!(μ, xla, B)
    mul!(μ, xla, B)
    BLAS.axpby!(1.0, μ_null, 1.0, μ)
    VCM_model = VCMTrait(Z, intercepts, xla, B, vc, μ)
    Y = Matrix(Transpose(simulate(VCM_model)))
    
    return xla, Matrix(Z'), B, Σ, Y
end

function simulate_NFBC1966_sparse(
    plinkname::String, k::Int, r::Int;
    seed::Int=2021, βoverlap=2, βσ = 1.0
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = SnpArray(plinkname * ".bed")
#     xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true)
    xla = convert(Matrix{Float64}, x, impute=true, center=true, scale=true)
    n, p = size(xla)
    
    # intercept is the only nongenetic covariate
    Z = ones(n, 1)
    intercepts = zeros(r)' # each trait have different intercept

    # simulate β
    B = simulate_random_beta(k, p, r, overlap=βoverlap, βσ=βσ)

    # between trait covariance matrix
    Σ = random_covariance_matrix(r)
    
    # simulate multivariate normal phenotype for each sample
    μ = xla * B + Z*intercepts

    # simulate response
    Y = zeros(n, r)
    for i in 1:n
        μi = @view(μ[i, :])
        Y[i, :] = rand(MvNormal(μi, Σ))
    end
    
    return xla, Matrix(Z'), B, Σ, Matrix(Y')
end

function simulate_multivariate_sparse(
    n::Int, p::Int, k::Int, r::Int;
    seed::Int=2021, βoverlap=2, 
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = simulate_random_snparray(undef, n, p)
    xla = convert(Matrix{Float64}, x, impute=true, center=true, scale=true)
#     xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true)
    n, p = size(x)

    # intercept is the only nongenetic covariate
    z = ones(n, 1)
    intercepts = zeros(r)' # each trait have different intercept

    # simulate response y, true model b, and the correct non-0 positions of b
    Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=βoverlap);
    
    return xla, Matrix(z'), true_b, true_Σ, Matrix(Y')
end

simulate_multivariate_sparse (generic function with 1 method)

# Try running IHT

In [26]:
# simulation parameters
k = 10
r = 2
βσ = 0.1
seed = 1111
βoverlap = 2

# use chr21 of NFBC
chr21 = "/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/qc/NFBC.qc.chr.21"
xla, Z, true_b, Σ, Y = simulate_NFBC1966_sparse(chr21,k,r,seed=seed,βoverlap=βoverlap,βσ=βσ)
causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

# simulate random data
# n = 1000
# p = 10000
# xla, Z, true_b, Σ, Y = simulate_multivariate_sparse(n, p, k,r,seed=seed,βoverlap=βoverlap)
# causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

In [27]:
println(Σ)
println(causal_snps)
true_b[causal_snps, :]

[0.056419962055242315 0.1802121298258207; 0.1802121298258207 2.723094788677102]
[2349, 3073, 3075, 4653, 5091, 5344, 5485, 2651]


8×2 Matrix{Float64}:
 -0.0523363   0.146389
 -0.105674    0.0
 -0.03193     0.0
  0.0654857   0.0
  0.0360225   0.0
 -0.0429316   0.0
  0.108861    0.0877691
  0.0        -0.141978

In [28]:
# no intercept term
function loglikelihood(X, Y, B, Γ)
    n = size(Y, 2)
    resid = Y - B*X
    return n/2 * logdet(Γ) - 0.5 * tr(Γ*resid*resid')
end

# this should be the best loglikehood attainable 
@show loglikelihood(Transpose(xla), Y, Transpose(true_b), inv(Σ))

# this should be loglikehood at iter 1 
@show loglikelihood(Transpose(xla), Y, zeros(size(Transpose(true_b))), [1.0 0.0; 0.0 1.0])

loglikelihood(Transpose(xla), Y, Transpose(true_b), inv(Σ)) = 373.8599630859753
loglikelihood(Transpose(xla), Y, zeros(size(Transpose(true_b))), [1.0 0.0; 0.0 1.0]) = -7480.311995286563


-7480.311995286563

In [37]:
result = fit_iht(Y, Transpose(xla), k=12, init_beta=true, verbose=true, debias=true)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior weight scaling = off
Doubly sparse projection = off
Debias = on
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -348.5823880579728, backtracks = 0, tol = 0.24348840335195368
Iteration 2: loglikelihood = -92.63716046636273, backtracks = 0, tol = 0.018033536901510928
Iteration 3: loglikelihood = -80.25529163986357, backtracks = 0, tol = 0.0
Iteration 4: loglikelihood = -78.93358707829248, backtracks = 0, tol = 0.0
Iteration 5: loglike


Compute time (sec):     0.6254458427429199
Final loglikelihood:    253.59208790828507
Iterations:             8
Trait 1's SNP PVE:      0.33214236400341335
Trait 2's SNP PVE:      0.015004585236065574

Estimated trait covariance:
2×2 DataFrame
 Row │ trait1     trait2   
     │ Float64    Float64  
─────┼─────────────────────
   1 │ 0.0580659  0.179454
   2 │ 0.179454   2.67415

Trait 1: IHT estimated 5 nonzero SNP predictors
5×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2349   -0.0502797
   2 │     3073   -0.0960889
   3 │     4653    0.0661157
   4 │     5091    0.0441988
   5 │     5485    0.107593

Trait 1: IHT estimated 0 non-genetic predictors
0×2 DataFrame

Trait 2: IHT estimated 7 nonzero SNP predictors
7×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2346    0.16039
   2 │     2348   -0.182616
   3 │     2349    0.130054
   4 │     2353    0.0

In [43]:
result = fit_iht(Y, Transpose(xla), k=12, verbose=true, debias=true)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior weight scaling = off
Doubly sparse projection = off
Debias = on
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1086.7080446381206, backtracks = 0, tol = 0.0


┌ Warning: Replacing docs for `MendelIHT.fit_iht! :: Union{Tuple{Union{IHTVariable{T, M}, MendelIHT.mIHTVariable{T, M}}}, Tuple{M}, Tuple{T}} where {T<:Union{Float32, Float64}, M}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.fit_iht! :: Union{Tuple{Union{IHTVariable{T, M}, MendelIHT.mIHTVariable{T, M}}}, Tuple{M}, Tuple{T}} where {T<:Union{Float32, Float64}, M}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.fit_iht! :: Union{Tuple{Union{IHTVariable{T, M}, MendelIHT.mIHTVariable{T, M}}}, Tuple{M}, Tuple{T}} where {T<:Union{Float32, Float64}, M}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.fit_iht! :: Union{Tuple{Union{IHTVariable{T, M}, MendelIHT.mIHTVariable{T, M}}}, Tuple{M}, Tuple{T}} where {T<:Union{Float32, Float64}, M}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.fit_iht! :: Union{Tuple

Iteration 2: loglikelihood = -1083.8182233107045, backtracks = 0, tol = 7.068275112573435e-5
Iteration 3: loglikelihood = -176.068152614067, backtracks = 0, tol = 0.035389719786740916
Iteration 4: loglikelihood = -47.66422231391789, backtracks = 0, tol = 0.025845269446551987
Iteration 5: loglikelihood = 176.3059362969816, backtracks = 1, tol = 0.017070492275814027
Iteration 6: loglikelihood = 239.97371844612735, backtracks = 1, tol = 0.08418738084591948
Iteration 7: loglikelihood = 355.59977418903236, backtracks = 0, tol = 0.027136988474549074
Iteration 8: loglikelihood = 324.99253598381165, backtracks = 3, tol = 0.017526955061516632
Iteration 9: loglikelihood = 336.9928024581186, backtracks = 0, tol = 0.0



Compute time (sec):     0.6988649368286133
Final loglikelihood:    355.59977418903236
Iterations:             9
Trait 1's SNP PVE:      0.39179817428213126
Trait 2's SNP PVE:      0.008613478375068375

Estimated trait covariance:
2×2 DataFrame
 Row │ trait1     trait2   
     │ Float64    Float64  
─────┼─────────────────────
   1 │ 0.0562832  0.179476
   2 │ 0.179476   2.69174

Trait 1: IHT estimated 7 nonzero SNP predictors
7×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2349   -0.0421664
   2 │     3073   -0.103315
   3 │     3077    0.0335914
   4 │     4653    0.0661619
   5 │     5091    0.04317
   6 │     5344   -0.0359141
   7 │     5485    0.105314

Trait 1: IHT estimated 0 non-genetic predictors
0×2 DataFrame

Trait 2: IHT estimated 5 nonzero SNP predictors
5×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2349    0.0898681
   2 │     2350    0.

In [44]:
# beta 1
[result.beta[1, causal_snps] true_b[causal_snps, 1]]

8×2 Matrix{Float64}:
 -0.0421664  -0.0523363
 -0.103315   -0.105674
  0.0        -0.03193
  0.0661619   0.0654857
  0.04317     0.0360225
 -0.0359141  -0.0429316
  0.105314    0.108861
  0.0         0.0

In [45]:
# beta 2
[result.beta[2, causal_snps] true_b[causal_snps, 2]]

8×2 Matrix{Float64}:
 0.0898681   0.146389
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0         0.0
 0.0265674   0.0877691
 0.0        -0.141978

In [46]:
# number of true positives
count(!iszero, sum(result.beta[:, causal_snps], dims=1))

6

In [81]:
# covariance
[vec(result.Σ) vec(Σ)]

4×2 Matrix{Float64}:
 0.0487976  0.0506762
 0.442851   0.467675
 0.442851   0.467675
 7.17377    7.60324

In [82]:
causal_snps

8-element Vector{Int64}:
  837
 2918
 6678
 7685
 9709
 9861
 6268
 8428

## Try 20 simulations (NFBC data)

We will compare

+ Number of true positives
+ Number of iteration until convergence
+ Best cross validated k

in scenario where we initialize beta values, versus not initializing. 

In [3]:
function bench(sim::Int)
    # simulation parameters
    k = 10
    r = 2
    βσ = 0.1
    seed = 10
    βoverlap = 2
    chr21 = "/Users/biona001/Benjamin_Folder/UCLA/research/stampeed/qc/NFBC.qc.chr.21"

    no_init_TP = zeros(sim)
    no_init_iter = zeros(sim)
    no_init_best_k = zeros(sim)
    yes_init_TP = zeros(sim)
    yes_init_iter = zeros(sim)
    yes_init_best_k = zeros(sim)

    # use chr21 of NFBC
    for i in 1:sim
        xla, Z, true_b, Σ, Y = simulate_NFBC1966_sparse(chr21,k,r,seed=i,βoverlap=βoverlap,βσ=βσ)
        causal_snps = unique([x[1] for x in findall(!iszero, true_b)]);

        # not initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, verbose=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true)
        no_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        no_init_iter[i] = result.iter
        no_init_best_k[i] = argmin(mses)

        # initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, init_beta=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, init_beta = true)
        yes_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        yes_init_iter[i] = result.iter
        yes_init_best_k[i] = argmin(mses)
    end

    return no_init_TP, no_init_iter, no_init_best_k, 
        yes_init_TP, yes_init_iter, yes_init_best_k
end
@time no_init_TP, no_init_iter, no_init_best_k, 
    yes_init_TP, yes_init_iter, yes_init_best_k = bench(20)

Cross validating...100%|████████████████████████████████| Time: 0:05:55




Crossvalidation Results:
	k	MSE
	1	3345.6904133447288
	2	3266.064026264423
	3	3308.514904028456
	4	3214.707484836648
	5	3197.9836884029924
	6	3482.6959584219576
	7	3184.2409542747946
	8	3207.369832866423
	9	3182.14486949898
	10	3238.694606270842
	11	3195.5329941447835
	12	3176.4583089321204
	13	3182.3111285965006
	14	3170.329129314194
	15	3178.609247029076
	16	3179.04562682311
	17	3161.6023182510444
	18	3165.2876753910646
	19	3174.51164010528
	20	3168.4513783499056

Best k = 17

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k

Cross validating...100%|████████████████████████████████| Time: 0:05:26




Crossvalidation Results:
	k	MSE
	1	3342.878914269393
	2	3263.737015594801
	3	6654.441546732709
	4	3210.553963108172
	5	3385.649355545925
	6	3472.985071199658
	7	3181.7746714981686
	8	3180.1537920046744
	9	3209.989773900449
	10	3180.2941991822913
	11	3210.2038331395843
	12	3163.883353699359
	13	3180.1074697076324
	14	3187.175164361818
	15	3215.1654189007727
	16	3165.931708210023
	17	3166.5662156449457
	18	3164.4594202871376
	19	3173.2040456111235
	20	3191.0968520382708

Best k = 12

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter

Cross validating...100%|████████████████████████████████| Time: 0:02:50


...

Cross validating...100%|████████████████████████████████| Time: 0:04:38




Crossvalidation Results:
	k	MSE
	1	6644.190189656684
	2	6323.768584039205
	3	7452.9545976593145
	4	6272.817304424122
	5	6233.153317099792
	6	6258.290721703608
	7	6248.967838467211
	8	6246.849622413312
	9	6284.385914605509
	10	6269.693526738198
	11	6238.486219804894
	12	6223.897496554079
	13	6240.122599148202
	14	6246.302730252291
	15	6242.16196448419
	16	6235.068049964559
	17	6238.37542686255
	18	6240.950815005985
	19	6291.791119394435
	20	6269.72581872072

Best k = 12

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Pr

Cross validating...100%|████████████████████████████████| Time: 0:05:04




Crossvalidation Results:
	k	MSE
	1	6626.493573383584
	2	6302.110982719853
	3	6261.2071583190955
	4	6204.32986733236
	5	6207.88945631343
	6	6193.3655131376745
	7	6200.027114569599
	8	6201.584678683672
	9	6233.961490828966
	10	6214.65216660369
	11	6247.132247754193
	12	6235.855513227682
	13	6223.046696783778
	14	6217.216137049305
	15	6248.165017746403
	16	6236.33358732366
	17	6238.680711779096
	18	6269.77893399597
	19	6248.889390166834
	20	6263.63578291394

Best k = 6

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior

([6.0, 8.0, 5.0, 5.0, 1.0, 7.0, 5.0, 5.0, 4.0, 4.0, 3.0, 4.0, 7.0, 5.0, 1.0, 0.0, 6.0, 5.0, 6.0, 4.0], [200.0, 67.0, 200.0, 200.0, 200.0, 200.0, 200.0, 43.0, 10.0, 7.0, 52.0, 60.0, 200.0, 200.0, 200.0, 5.0, 78.0, 14.0, 200.0, 200.0], [17.0, 20.0, 17.0, 12.0, 10.0, 20.0, 10.0, 15.0, 6.0, 6.0, 10.0, 8.0, 16.0, 12.0, 5.0, 2.0, 19.0, 9.0, 14.0, 12.0], [5.0, 8.0, 0.0, 5.0, 6.0, 4.0, 5.0, 5.0, 8.0, 6.0, 4.0, 4.0, 6.0, 5.0, 3.0, 4.0, 6.0, 6.0, 6.0, 4.0], [200.0, 81.0, 200.0, 200.0, 134.0, 75.0, 200.0, 96.0, 13.0, 200.0, 30.0, 44.0, 200.0, 200.0, 200.0, 200.0, 79.0, 17.0, 200.0, 200.0], [12.0, 5.0, 8.0, 9.0, 9.0, 8.0, 7.0, 12.0, 6.0, 6.0, 10.0, 8.0, 13.0, 14.0, 3.0, 2.0, 17.0, 9.0, 9.0, 6.0])

In [7]:
@show mean(no_init_TP)
@show mean(yes_init_TP)
[no_init_TP yes_init_TP]

mean(no_init_TP) = 4.55
mean(yes_init_TP) = 5.0


20×2 Matrix{Float64}:
 6.0  5.0
 8.0  8.0
 5.0  0.0
 5.0  5.0
 1.0  6.0
 7.0  4.0
 5.0  5.0
 5.0  5.0
 4.0  8.0
 4.0  6.0
 3.0  4.0
 4.0  4.0
 7.0  6.0
 5.0  5.0
 1.0  3.0
 0.0  4.0
 6.0  6.0
 5.0  6.0
 6.0  6.0
 4.0  4.0

In [8]:
@show mean(no_init_iter)
@show mean(yes_init_iter)
[no_init_iter yes_init_iter]


mean(no_init_iter) = 126.8
mean(yes_init_iter) = 138.45


20×2 Matrix{Float64}:
 200.0  200.0
  67.0   81.0
 200.0  200.0
 200.0  200.0
 200.0  134.0
 200.0   75.0
 200.0  200.0
  43.0   96.0
  10.0   13.0
   7.0  200.0
  52.0   30.0
  60.0   44.0
 200.0  200.0
 200.0  200.0
 200.0  200.0
   5.0  200.0
  78.0   79.0
  14.0   17.0
 200.0  200.0
 200.0  200.0

In [9]:
@show mean(no_init_best_k)
@show mean(yes_init_best_k)
[no_init_best_k yes_init_best_k]

mean(no_init_best_k) = 12.0
mean(yes_init_best_k) = 8.65


20×2 Matrix{Float64}:
 17.0  12.0
 20.0   5.0
 17.0   8.0
 12.0   9.0
 10.0   9.0
 20.0   8.0
 10.0   7.0
 15.0  12.0
  6.0   6.0
  6.0   6.0
 10.0  10.0
  8.0   8.0
 16.0  13.0
 12.0  14.0
  5.0   3.0
  2.0   2.0
 19.0  17.0
  9.0   9.0
 14.0   9.0
 12.0   6.0

## Try 20 simulations (simulated genotypes)

We will compare

+ Number of true positives
+ Number of iteration until convergence
+ Best cross validated k

in scenario where we initialize beta values, versus not initializing. 

In [51]:
function bench(sim::Int)
    # simulation parameters
    n = 1000
    p = 10000
    k = 10
    r = 2
    βσ = 0.1
    βoverlap = 0

    no_init_TP = zeros(sim)
    no_init_iter = zeros(sim)
    no_init_best_k = zeros(sim)
    yes_init_TP = zeros(sim)
    yes_init_iter = zeros(sim)
    yes_init_best_k = zeros(sim)

    for i in 1:sim
        xla, Z, true_b, Σ, Y = simulate_multivariate_sparse(n, p, k,r,seed=i,βoverlap=βoverlap)
        causal_snps = unique([x[1] for x in findall(!iszero, true_b)])

        # not initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, verbose=true, debias=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, debias=true)
        no_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        no_init_iter[i] = result.iter
        no_init_best_k[i] = argmin(mses)

        # initializing beta
        mses = cv_iht(Y, Transpose(xla), path=1:20, init_beta=true, debias=true)
        result = fit_iht(Y, Transpose(xla), k=argmin(mses), verbose=true, init_beta = true, debias=true)
        yes_init_TP[i] = count(!iszero, sum(result.beta[:, causal_snps], dims=1))
        yes_init_iter[i] = result.iter
        yes_init_best_k[i] = argmin(mses)
    end

    return no_init_TP, no_init_iter, no_init_best_k, 
        yes_init_TP, yes_init_iter, yes_init_best_k
end
@time no_init_TP, no_init_iter, no_init_best_k, 
    yes_init_TP, yes_init_iter, yes_init_best_k = bench(20)

Cross validating...100%|████████████████████████████████| Time: 0:00:15




Crossvalidation Results:
	k	MSE
	1	3054.793264535042
	2	3056.3025981493015
	3	1385.711463791572
	4	826.9266005988334
	5	380.7077275030195
	6	285.15270616414927
	7	204.6031797484711
	8	133.7885163523987
	9	124.53754284424802
	10	125.37175558572136
	11	126.41997609398638
	12	123.89158338622163
	13	125.80471478106737
	14	126.75551440247028
	15	126.86603492736293
	16	129.20003074764134
	17	127.65346578207303
	18	128.52755513237457
	19	128.4157521947696
	20	130.262818111156

Best k = 12

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity paramete

Cross validating...100%|████████████████████████████████| Time: 0:00:23




Crossvalidation Results:
	k	MSE
	1	2473.744641162287
	2	1919.7695355012788
	3	1317.20922785413
	4	792.3897862079906
	5	379.9714410550192
	6	289.69404287594557
	7	201.89947111325324
	8	133.9479264279291
	9	124.76775416400523
	10	124.52811014708195
	11	124.34615657946154
	12	124.12115316781934
	13	125.48194803186833
	14	126.03742157043308
	15	125.51817156761408
	16	126.30416781908514
	17	126.95891248072653
	18	126.9935833061806
	19	126.8552408723718
	20	127.1870711317012

Best k = 12

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity paramete

Cross validating...100%|████████████████████████████████| Time: 0:00:09


...

Cross validating...100%|████████████████████████████████| Time: 0:00:25




Crossvalidation Results:
	k	MSE
	1	2990.8474096892164
	2	2992.3088051990553
	3	1494.4717682845976
	4	1341.534072974977
	5	1332.648046300467
	6	1334.6513808691127
	7	1333.110226331015
	8	1330.5466495974354
	9	1328.4768057415151
	10	1337.4089666182967
	11	1334.1459974519678
	12	1336.912620283119
	13	1331.078924590034
	14	1333.87966276255
	15	1341.988039310459
	16	1334.9788577561853
	17	1342.0987475741301
	18	1337.6766283881086
	19	1354.5854036401154
	20	1340.6160370775158

Best k = 9

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity paramete

Cross validating...100%|████████████████████████████████| Time: 0:00:34




Crossvalidation Results:
	k	MSE
	1	1996.313263237039
	2	1689.8554332635886
	3	1470.7049120019306
	4	1344.0057838453026
	5	1335.0150882759845
	6	1337.1938936153776
	7	1335.3770454500113
	8	1332.257798533526
	9	1332.4252532655528
	10	1330.9335550806786
	11	1340.098699468186
	12	1332.0911905742512
	13	1332.7314445059524
	14	1326.755867186044
	15	1334.2289438042749
	16	1341.185598080644
	17	1332.36385948852
	18	1335.0200704103552
	19	1341.4319954894136
	20	1341.1683986073658

Best k = 14

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parame

([9.0, 9.0, 9.0, 10.0, 7.0, 8.0, 9.0, 9.0, 10.0, 10.0, 10.0, 8.0, 10.0, 10.0, 8.0, 7.0, 9.0, 7.0, 7.0, 6.0], [10.0, 6.0, 21.0, 11.0, 6.0, 8.0, 6.0, 6.0, 6.0, 8.0, 8.0, 10.0, 11.0, 6.0, 7.0, 200.0, 8.0, 6.0, 6.0, 11.0], [12.0, 10.0, 18.0, 19.0, 7.0, 10.0, 9.0, 9.0, 11.0, 10.0, 17.0, 9.0, 19.0, 10.0, 9.0, 12.0, 11.0, 7.0, 7.0, 9.0], [9.0, 10.0, 9.0, 9.0, 7.0, 8.0, 10.0, 9.0, 10.0, 10.0, 10.0, 8.0, 8.0, 10.0, 8.0, 7.0, 9.0, 7.0, 7.0, 7.0], [10.0, 13.0, 8.0, 12.0, 6.0, 200.0, 7.0, 200.0, 6.0, 7.0, 9.0, 6.0, 8.0, 6.0, 6.0, 200.0, 8.0, 6.0, 7.0, 11.0], [12.0, 18.0, 18.0, 20.0, 7.0, 13.0, 16.0, 12.0, 11.0, 12.0, 16.0, 8.0, 10.0, 10.0, 8.0, 11.0, 12.0, 7.0, 7.0, 14.0])

In [52]:
@show mean(no_init_TP)
@show mean(yes_init_TP)
[no_init_TP yes_init_TP]

mean(no_init_TP) = 8.6
mean(yes_init_TP) = 8.6


20×2 Matrix{Float64}:
  9.0   9.0
  9.0  10.0
  9.0   9.0
 10.0   9.0
  7.0   7.0
  8.0   8.0
  9.0  10.0
  9.0   9.0
 10.0  10.0
 10.0  10.0
 10.0  10.0
  8.0   8.0
 10.0   8.0
 10.0  10.0
  8.0   8.0
  7.0   7.0
  9.0   9.0
  7.0   7.0
  7.0   7.0
  6.0   7.0

In [53]:
@show mean(no_init_iter)
@show mean(yes_init_iter)
[no_init_iter yes_init_iter]

mean(no_init_iter) = 18.05
mean(yes_init_iter) = 36.8


20×2 Matrix{Float64}:
  10.0   10.0
   6.0   13.0
  21.0    8.0
  11.0   12.0
   6.0    6.0
   8.0  200.0
   6.0    7.0
   6.0  200.0
   6.0    6.0
   8.0    7.0
   8.0    9.0
  10.0    6.0
  11.0    8.0
   6.0    6.0
   7.0    6.0
 200.0  200.0
   8.0    8.0
   6.0    6.0
   6.0    7.0
  11.0   11.0

In [54]:
@show mean(no_init_best_k)
@show mean(yes_init_best_k)
[no_init_best_k yes_init_best_k]

mean(no_init_best_k) = 11.25
mean(yes_init_best_k) = 12.1


20×2 Matrix{Float64}:
 12.0  12.0
 10.0  18.0
 18.0  18.0
 19.0  20.0
  7.0   7.0
 10.0  13.0
  9.0  16.0
  9.0  12.0
 11.0  11.0
 10.0  12.0
 17.0  16.0
  9.0   8.0
 19.0  10.0
 10.0  10.0
  9.0   8.0
 12.0  11.0
 11.0  12.0
  7.0   7.0
  7.0   7.0
  9.0  14.0

In [48]:
@show mean(no_init_TP)
@show mean(yes_init_TP)
[no_init_TP yes_init_TP]

mean(no_init_TP) = 8.4
mean(yes_init_TP) = 8.45


┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tuple{MendelIHT.mIHTVariable}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `MendelIHT.score! :: Tupl

20×2 Matrix{Float64}:
  9.0   9.0
  9.0   9.0
  9.0   9.0
  8.0  10.0
  7.0   7.0
  6.0   8.0
  9.0  10.0
  9.0   9.0
 10.0  10.0
 10.0  10.0
 10.0  10.0
  8.0   8.0
  9.0   8.0
 10.0  10.0
  8.0   7.0
  8.0   7.0
  9.0   9.0
  7.0   7.0
  7.0   7.0
  6.0   5.0

In [49]:
@show mean(no_init_iter)
@show mean(yes_init_iter)
[no_init_iter yes_init_iter]

mean(no_init_iter) = 53.95
mean(yes_init_iter) = 44.05


20×2 Matrix{Float64}:
  19.0   13.0
  22.0   20.0
 167.0   74.0
  26.0   42.0
  14.0   10.0
  11.0    8.0
  42.0   32.0
 200.0  200.0
  14.0   12.0
  21.0   47.0
  25.0   13.0
  11.0    8.0
 200.0  145.0
  11.0    8.0
   7.0    5.0
 200.0  200.0
  49.0   16.0
  10.0    7.0
  10.0   10.0
  20.0   11.0

In [50]:
@show mean(no_init_best_k)
@show mean(yes_init_best_k)
[no_init_best_k yes_init_best_k]

mean(no_init_best_k) = 9.05
mean(yes_init_best_k) = 9.65


20×2 Matrix{Float64}:
  9.0   9.0
  9.0   9.0
 11.0  13.0
 12.0  16.0
  7.0   7.0
  6.0   8.0
  9.0  12.0
  9.0   9.0
 10.0  11.0
 10.0  13.0
 10.0  10.0
  8.0   8.0
  9.0  10.0
 10.0  10.0
  8.0   7.0
 12.0  11.0
 12.0  10.0
  7.0   7.0
  7.0   7.0
  6.0   6.0

## Can we increase IHT's true positive by finding nearby SNPs?

Short answer: not really (increase is 1-2%)

In [29]:
TP, TP_LD = process_iht(0.9, verbose=false)
@show mean(TP), mean(TP_LD);

k = 10, r = 2, polygenic model

(mean(TP), mean(TP_LD)) = (0.42875, 0.43875)


In [30]:
TP, TP_LD = process_iht(0.5)
@show mean(TP), mean(TP_LD);

k = 10, r = 2, polygenic model

(mean(TP), mean(TP_LD)) = (0.42875, 0.44125)


## Conclusion:
+ mvPLINK and GEMMA have essentially identical power, about 20% higher than IHT
+ High power of mvPLINK and GEMMA also gives **extremely high** false positives (type I error)
+ QQ and Manhattan plot for GEMMA and PLINK often look insanely bad
+ **Simulated genotypes *does not* produce insane p-values for GEMMA and mvPLINK.**
+ Searching SNPs in LD with the causal SNPs does not increase true positive rate for IHT